### Dependências

In [1]:
from pymongo import *
from pprint import *

### Conexão

In [2]:
client = MongoClient('localhost:27017') #A porta padrão do mongodb é a 27017.

### Criação do Database

In [3]:
db = client["amazon-meta"]

### Execução do parser.py antes da inserção da coleção de dados no Database

In [4]:
!python3 parser.py > amazon.json  #O path do arquivo "amazon-meta.txt.gz" deve está no mesmo diretório, assim como está especificado no arquivo "parser.py" 

100%|███████████████████████████| 15010574/15010574 [01:18<00:00, 190301.35it/s]


#### Utilizaremos os dados do arquivo 'amazon.json', na qual foi gerado através do "parser.py". Os dados foram inseridos no Mongo DB através do seguinte comando:

In [5]:
!mongoimport --host localhost --port 27017 --collection amazon --db amazon-meta --file amazon.json

2020-02-27T20:57:53.818-0400	connected to: localhost:27017
2020-02-27T20:57:56.708-0400	[........................] amazon-meta.amazon	22.7MB/1.03GB (2.2%)
2020-02-27T20:57:59.708-0400	[#.......................] amazon-meta.amazon	61.4MB/1.03GB (5.8%)
2020-02-27T20:58:02.708-0400	[##......................] amazon-meta.amazon	94.6MB/1.03GB (9.0%)
2020-02-27T20:58:05.708-0400	[##......................] amazon-meta.amazon	132MB/1.03GB (12.5%)
2020-02-27T20:58:08.708-0400	[###.....................] amazon-meta.amazon	158MB/1.03GB (15.0%)
2020-02-27T20:58:11.709-0400	[####....................] amazon-meta.amazon	192MB/1.03GB (18.1%)
2020-02-27T20:58:14.708-0400	[#####...................] amazon-meta.amazon	233MB/1.03GB (22.1%)
2020-02-27T20:58:17.708-0400	[######..................] amazon-meta.amazon	268MB/1.03GB (25.3%)
2020-02-27T20:58:20.708-0400	[######..................] amazon-meta.amazon	290MB/1.03GB (27.5%)
2020-02-27T20:58:23.708-0400	[#######.................] amazon-meta.amazon	32

### Consultas

#### (a) Dado produto, listar os 5 comentários mais úteis e com maior avaliação e os 5 comentários mais úteis e com menor avaliação

In [6]:
produto = str(input("Escolha o id de qual produto deseja: "))
print()
a_maior = [{"$match":{ "id" : produto } },{"$unwind": "$reviews"},{"$sort": {"reviews.helpful": -1, "reviews.rating": -1}},{"$project": {"helpful": "$reviews.helpful", "rating":"$reviews.rating"}},{"$limit": 5}]
a_menor = [{"$match":{ "id" : produto } },{"$unwind": "$reviews"},{"$sort": {"reviews.rating": 1, "reviews.helpful": -1 }},{"$project": {"helpful": "$reviews.helpful", "rating":"$reviews.rating"}},{"$limit": 5}]

pprint(list(db.amazon.aggregate(a_maior)))
pprint(list(db.amazon.aggregate(a_menor)))

Escolha o id de qual produto deseja: 2

[{'_id': ObjectId('5e586592714c44129af91c19'), 'helpful': 16, 'rating': 4},
 {'_id': ObjectId('5e586592714c44129af91c19'), 'helpful': 9, 'rating': 1},
 {'_id': ObjectId('5e586592714c44129af91c19'), 'helpful': 8, 'rating': 5},
 {'_id': ObjectId('5e586592714c44129af91c19'), 'helpful': 8, 'rating': 5},
 {'_id': ObjectId('5e586592714c44129af91c19'), 'helpful': 6, 'rating': 4}]
[{'_id': ObjectId('5e586592714c44129af91c19'), 'helpful': 9, 'rating': 1},
 {'_id': ObjectId('5e586592714c44129af91c19'), 'helpful': 16, 'rating': 4},
 {'_id': ObjectId('5e586592714c44129af91c19'), 'helpful': 6, 'rating': 4},
 {'_id': ObjectId('5e586592714c44129af91c19'), 'helpful': 5, 'rating': 4},
 {'_id': ObjectId('5e586592714c44129af91c19'), 'helpful': 5, 'rating': 4}]


#### (b) Dado um produto, listar os produtos similares com maiores vendas do que ele

In [7]:
id_produto = str(input("Escolha o id de qual produto deseja: "))
print()
produto = db.amazon.find_one({"id": id_produto })

print("Produto dado: " + produto["title"] + ", " + "salenrank: " +  produto["salesrank"])

print()

querry = {"ASIN":{"$in": produto["similar_items"]}, "$and":[{"salesrank": {"$lt": produto["salesrank"]} }]}

b = db.amazon.find(querry)
print("Os produtos similares com maiores vendas do que ele: ")
for result in b:
    print("title: " + result["title"] + "," + " salesrank: " + result["salesrank"])
    

Escolha o id de qual produto deseja: 2

Produto dado: Candlemas: Feast of Flames, salenrank: 168596

Os produtos similares com maiores vendas do que ele: 
title: Yule: A Celebration of Light and Warmth, salesrank: 103012
title: Midsummer: Magical Celebrations of the Summer Solstice, salesrank: 159277


#### (c) Dado um produto, mostrar a evolução diária das médias de avaliação ao longo do intervalo de tempo coberto no arquivo de entrada

In [8]:
produto = str(input("Escolha o id de qual produto deseja: "))
print()
c = [{"$match":{ "id" : produto } },{"$unwind": "$reviews"},{"$group": {"_id": "$reviews._date", "avgRating": {"$avg": "$reviews.rating"}}},{"$sort": {"_id": 1}}]
pprint(list(db.amazon.aggregate(c)))

Escolha o id de qual produto deseja: 2

[{'_id': '2001-12-16', 'avgRating': 5.0},
 {'_id': '2002-1-24', 'avgRating': 5.0},
 {'_id': '2002-1-28', 'avgRating': 5.0},
 {'_id': '2002-1-7', 'avgRating': 4.0},
 {'_id': '2002-2-14', 'avgRating': 4.0},
 {'_id': '2002-2-6', 'avgRating': 4.0},
 {'_id': '2002-3-23', 'avgRating': 4.0},
 {'_id': '2002-5-23', 'avgRating': 5.0},
 {'_id': '2003-11-25', 'avgRating': 5.0},
 {'_id': '2003-2-25', 'avgRating': 5.0},
 {'_id': '2004-2-11', 'avgRating': 1.0},
 {'_id': '2005-2-7', 'avgRating': 5.0}]


#### (d) Listar os 10 produtos lideres de venda em cada grupo de produtos

#### (e) Listar os 10 produtos com a maior média de avaliações úteis positivas por produto

In [9]:
e = [{"$unwind": "$reviews"},{"$group": {"_id": "$title", "avgHelpful": {"$avg": '$reviews.helpful'}}},{"$sort": {"avgHelpful": -1}},{"$limit": 10}]
pprint(list(db.amazon.aggregate(e)))

[{'_id': 'Understanding Loved Boys and Boylovers', 'avgHelpful': 243.0},
 {'_id': "T'ai Chi for Older Adults", 'avgHelpful': 233.0},
 {'_id': 'More Than Just Hot Air: Common Sense Counter-Top Convection Oven '
         'Cookbook',
  'avgHelpful': 203.0},
 {'_id': 'Crockpot Cookery (Cookbooks By Morris Press)', 'avgHelpful': 197.0},
 {'_id': 'Creative Interventions for Troubled Children & Youth',
  'avgHelpful': 196.0},
 {'_id': 'The Story About Ping', 'avgHelpful': 186.46511627906978},
 {'_id': 'The Story About Ping (8x 8)', 'avgHelpful': 186.34883720930233},
 {'_id': 'The Story about Ping : StoryTape (StoryTape, Puffin)',
  'avgHelpful': 186.32558139534885},
 {'_id': 'The Story about Ping', 'avgHelpful': 186.30232558139534},
 {'_id': 'The Smoked-Foods Cookbook: How to Flavor, Cure, and Prepare Savory '
         'Meats, Game, Fish, Nuts, and Cheese',
  'avgHelpful': 183.0}]


#### (f) Listar a 5 categorias de produto com a maior média de avaliações úteis positivas por produto

In [10]:
f = [{"$unwind": "$reviews"},{"$unwind": "$categories"},{"$group": {"_id": "$categories", "avgHelpful": {"$avg": '$reviews.helpful'}}},{"$sort": {"avgHelpful": -1}},{"$limit": 5}]
pprint(list(db.amazon.aggregate(f)))

[{'_id': '|Books[283155]|Subjects[1000]|Computers & '
         'Internet[5]|Software[4053]|Personal Finance[4169]|Intuit '
         'Quicken[4171]|Casual Users[502030]',
  'avgHelpful': 84.5},
 {'_id': '|[229534]|Specialty Stores[509348]|Software '
         'Books[727676]|Business[727678]|Intuit Quicken[727728]|Casual '
         'Users[727732]',
  'avgHelpful': 84.5},
 {'_id': '|Books[283155]|Subjects[1000]|History[9]|Americas[4808]|Caribbean & '
         'West Indies[4810]|Dominican Republic[4816]',
  'avgHelpful': 78.69491525423729},
 {'_id': '|Music[5174]|Styles[301668]|Classical[85]|Featured Composers, '
         'A-Z[5338]|( G )[19383]|Goetz, Hermann[20197]',
  'avgHelpful': 75.0},
 {'_id': '|[229534]|Categories[491286]', 'avgHelpful': 70.0}]


#### (g) Listar os 10 clientes que mais fizeram comentários por grupo de produto